# OpenAI Agents - 도구 고급

이 튜토리얼은 OpenAI Agents SDK에서 에이전트에게 도구를 제공하는 방법을 다룬다. 기본 함수 도구 생성부터 OpenAI가 제공하는 호스팅 도구 사용까지 핵심 패턴을 학습한다.

In [ ]:
# 필요한 패키지 설치
!pip install openai-agents python-dotenv pydantic

---

## 1. 기본 함수 도구

이 챕터에서는 에이전트를 위한 기본 함수 도구를 생성하는 방법을 다룬다.

### 학습 내용

- @function_tool 데코레이터를 사용한 도구 생성
- Pydantic 모델을 활용한 구조화된 데이터 반환
- 에이전트당 여러 도구 사용
- 독스트링을 통한 도구 문서화

### 환경 설정

In [1]:
import os
import asyncio
from dotenv import load_dotenv
from pydantic import BaseModel
from typing import List
from agents import Agent, Runner, function_tool

# 환경 변수 로드
load_dotenv()

MODEL = "gpt-4o-mini"

### 구조화된 데이터 모델 정의

도구가 반환할 구조화된 데이터를 위한 Pydantic 모델을 정의한다.

In [2]:
class WeatherData(BaseModel):
    """
    날씨 정보를 담는 구조화된 모델이다.
    """
    city: str                  # 도시 이름
    temperature: float         # 온도 (섭씨)
    conditions: str            # 날씨 상태
    humidity: int              # 습도 (%)
    forecast: List[str]        # 예보 리스트


print("WeatherData 모델이 정의되었다.")
print(f"필드: city, temperature, conditions, humidity, forecast")

WeatherData 모델이 정의되었다.
필드: city, temperature, conditions, humidity, forecast


### @function_tool로 도구 생성

@function_tool 데코레이터를 사용하여 에이전트가 호출할 수 있는 도구를 만든다. 독스트링이 자동으로 도구 설명으로 사용된다.

In [3]:
@function_tool
def get_current_weather(city: str) -> WeatherData:
    """
    지정된 도시의 현재 날씨 상태를 가져온다.
    
    Args:
        city: 날씨를 조회할 도시 이름
        
    Returns:
        온도와 상태를 포함한 현재 날씨 데이터
    """
    # 모의 날씨 데이터 - 실제 구현에서는 날씨 API 호출
    weather_db = {
        "서울": WeatherData(
            city="서울",
            temperature=18.5,
            conditions="맑음",
            humidity=65,
            forecast=["내일 구름 조금", "주말 화창"]
        ),
        "new york": WeatherData(
            city="New York",
            temperature=18.5,
            conditions="Cloudy",
            humidity=72,
            forecast=["내일 가벼운 비", "주말 맑음"]
        ),
        "london": WeatherData(
            city="London", 
            temperature=12.0,
            conditions="Rainy",
            humidity=85,
            forecast=["내일 흐림", "금요일부터 개임"]
        ),
        "tokyo": WeatherData(
            city="Tokyo",
            temperature=25.3,
            conditions="Sunny",
            humidity=58,
            forecast=["내일 덥고 습함", "뇌우 가능성"]
        ),
        "부산": WeatherData(
            city="부산",
            temperature=22.0,
            conditions="구름 조금",
            humidity=70,
            forecast=["내일 맑음", "주말 소나기 가능"]
        )
    }
    
    return weather_db.get(city.lower(), WeatherData(
        city=city,
        temperature=20.0,
        conditions="데이터 없음",
        humidity=50,
        forecast=["예보 없음"]
    ))


print("get_current_weather 도구가 정의되었다.")

get_current_weather 도구가 정의되었다.


In [4]:
@function_tool
def get_weather_alerts(city: str) -> List[str]:
    """
    도시의 날씨 경보와 주의보를 가져온다.
    
    Args:
        city: 경보를 확인할 도시 이름
        
    Returns:
        활성 날씨 경보 리스트
    """
    # 모의 경보 데이터
    alerts_db = {
        "서울": ["미세먼지 주의보 발령"],
        "부산": ["태풍 주의보 - 내일 오후까지"],
        "miami": ["허리케인 감시 중", "오후 6시까지 홍수 경보"],
        "denver": ["폭설 주의보 - 3-6인치 예상"],
        "phoenix": ["극심한 폭염 경보 - 기온 43°C 이상"]
    }
    
    return alerts_db.get(city.lower(), [])


print("get_weather_alerts 도구가 정의되었다.")

get_weather_alerts 도구가 정의되었다.


### 여러 도구를 가진 에이전트 생성

여러 도구를 사용하는 날씨 에이전트를 생성한다.

In [5]:
# 도구를 가진 날씨 에이전트 생성
weather_agent = Agent(
    name="Weather Assistant",
    instructions="""
    당신은 유용한 날씨 어시스턴트입니다. 사용 가능한 도구를 사용하여 
    사용자가 묻는 모든 도시에 대해 정확하고 최신의 날씨 정보를 제공하세요.
    
    사용 가능한 도구:
    - get_current_weather: 현재 상태와 예보 조회
    - get_weather_alerts: 날씨 경보와 주의보 확인
    
    항상 현재 상태와 경보를 포함한 포괄적인 날씨 정보를 제공하세요.
    한국어로 응답하세요.
    """,
    tools=[get_current_weather, get_weather_alerts],
)

print(f"'{weather_agent.name}' 에이전트가 생성되었다.")
print(f"도구 수: {len(weather_agent.tools)}개")

'Weather Assistant' 에이전트가 생성되었다.
도구 수: 2개


### 기본 도구 데모 실행

In [6]:
async def demo_basic_tools():
    """
    구조화된 데이터와 기본 함수 도구를 시연한다.
    """
    
    queries = [
        "서울 날씨 어때요?",
        "부산에 날씨 경보가 있나요?", 
        "도쿄 날씨와 경보를 알려주세요",
        "런던 예보는 어떤가요?",
        "피닉스 날씨와 경보 확인해주세요"
    ]
    
    print("🛠️ 기본 도구 데모")
    print("구조화된 데이터와 여러 도구 사용을 보여준다")
    print("=" * 70)
    
    for i, query in enumerate(queries, 1):
        print(f"\n{i}️⃣ 질문: {query}")
        print("-" * 50)
        
        result = await Runner.run(weather_agent, query)
        print(f"응답: {result.final_output}")
        print("-" * 50)

# 데모 실행
await demo_basic_tools()

🛠️ 기본 도구 데모
구조화된 데이터와 여러 도구 사용을 보여준다

1️⃣ 질문: 서울 날씨 어때요?
--------------------------------------------------
응답: 현재 서울의 날씨는 맑으며, 기온은 18.5℃입니다. 습도는 65%로 쾌적한 상태입니다.

예보를 보면 내일은 구름이 약간 끼겠고, 주말에는 화창할 것으로 예상됩니다.

단, 현재 서울에는 미세먼지 주의보가 발령되어 있으니 외출 시 마스크 착용 등 건강에 유의하시기 바랍니다.
--------------------------------------------------

2️⃣ 질문: 부산에 날씨 경보가 있나요?
--------------------------------------------------
응답: 현재 부산에는 "태풍 주의보"가 발효 중입니다. 주의보는 내일 오후까지 이어질 예정이니, 외출 시 각별히 유의하시고 최신 기상 정보를 확인하시길 바랍니다.
--------------------------------------------------

3️⃣ 질문: 도쿄 날씨와 경보를 알려주세요
--------------------------------------------------
응답: 도쿄의 현재 날씨는 다음과 같습니다:
- 기온: 25.3℃
- 상태: 맑음 (Sunny)
- 습도: 58%
- 내일은 덥고 습하며, 뇌우가 올 가능성이 있습니다.

현재 도쿄 지역에 발효 중인 기상 경보와 주의보 정보도 곧 제공해 드리겠습니다. 잠시만 기다려 주세요.
--------------------------------------------------

4️⃣ 질문: 런던 예보는 어떤가요?
--------------------------------------------------
응답: 런던의 현재 날씨는 다음과 같습니다:
- 온도: 12°C
- 상태: 비가 오고 있습니다.
- 습도: 85%
- 예보: 내일은 흐림, 금요일부터 개일 예정입니다.

현재 발령된 날씨 경보와 

---

## 2. OpenAI 호스팅 도구

이 챕터에서는 OpenAI가 제공하는 호스팅 도구를 사용하는 방법을 다룬다.

### 학습 내용

- WebSearchTool을 활용한 인터넷 검색 기능
- 매개변수를 통한 호스팅 도구 구성
- OpenAI 서버에서 실행되는 도구 사용
- 실시간 정보 검색

In [8]:
from agents import WebSearchTool

### WebSearchTool 이해하기

WebSearchTool은 OpenAI 서버에서 실행되는 호스팅 도구로, 에이전트가 인터넷을 검색할 수 있게 해준다.

#### 📚 WebSearchTool 구성 옵션
==================================================

user_location: 검색 위치 설정
  - type: 'approximate' 또는 'precise'
  - city: 도시 이름

search_context_size: 검색 컨텍스트 크기
  - 'low': 적은 컨텍스트, 빠른 응답
  - 'medium': 중간 수준 (기본값)
  - 'high': 많은 컨텍스트, 상세 결과

### WebSearchTool을 가진 에이전트 생성

In [10]:
# 웹 검색 도구를 가진 리서치 에이전트
research_agent = Agent(
    name="Research Assistant",
    instructions="""
    당신은 사용자가 최신의 정확한 정보를 찾도록 돕는 리서치 어시스턴트입니다.
    항상 가장 최신 정보를 검색하고 출처를 명확하게 인용하세요.
    한국어로 응답하세요.
    """,
    tools=[
        WebSearchTool(
            user_location={"type": "approximate", "city": "Seoul"},
            search_context_size="medium"
        )
    ],
)

print(f"'{research_agent.name}' 에이전트가 생성되었다.")
print("  위치: Seoul")
print("  컨텍스트 크기: medium")

'Research Assistant' 에이전트가 생성되었다.
  위치: Seoul
  컨텍스트 크기: medium


In [11]:
# 다른 검색 구성을 가진 뉴스 에이전트
news_agent = Agent(
    name="News Reporter",
    instructions="""
    당신은 최신 뉴스에 집중하는 뉴스 리포터입니다.
    최신 뉴스를 검색하고 적절한 출처 인용과 함께 
    사실적이고 편향되지 않은 보도를 제공하세요.
    한국어로 응답하세요.
    """,
    tools=[
        WebSearchTool(
            user_location={"type": "approximate", "city": "London"},
            search_context_size="low"  # 빠른 응답을 위해 낮은 컨텍스트
        )
    ],
)

print(f"'{news_agent.name}' 에이전트가 생성되었다.")
print("  위치: London")
print("  컨텍스트 크기: low")

'News Reporter' 에이전트가 생성되었다.
  위치: London
  컨텍스트 크기: low


### 호스팅 도구 데모 실행

In [12]:
async def demo_hosted_tools():
    """
    웹 검색을 포함한 호스팅 도구를 시연한다.
    """
    
    queries = [
        ("research", "양자 컴퓨팅의 최신 발전은 무엇인가요?"),
        ("research", "전 세계 재생 에너지 도입 현황은 어떤가요?"),
        ("news", "인공지능 규제에 관한 최신 뉴스는 무엇인가요?"),
        ("research", "의료 AI 응용 분야의 최근 혁신은 무엇인가요?"),
        ("news", "이번 주 기후 변화 정책에 어떤 일이 있나요?")
    ]
    
    print("🌐 호스팅 도구 데모")
    print("OpenAI WebSearchTool을 사용한 실시간 정보 검색")
    print("=" * 70)
    
    for i, (agent_type, query) in enumerate(queries, 1):
        agent = research_agent if agent_type == "research" else news_agent
        
        print(f"\n{i}️⃣ 에이전트: {agent.name}")
        print(f"질문: {query}")
        print("-" * 50)
        
        try:
            result = await Runner.run(agent, query)
            # 응답이 길 경우 앞부분만 표시
            response = result.final_output
            if len(response) > 500:
                print(f"응답: {response[:500]}...")
            else:
                print(f"응답: {response}")
        except Exception as e:
            print(f"에러: {str(e)}")
        
        print("-" * 50)

# 데모 실행
await demo_hosted_tools()

🌐 호스팅 도구 데모
OpenAI WebSearchTool을 사용한 실시간 정보 검색

1️⃣ 에이전트: Research Assistant
질문: 양자 컴퓨팅의 최신 발전은 무엇인가요?
--------------------------------------------------
응답: 아래는 2025년 12월 12일(금, 오늘 기준)까지 확인된 **양자 컴퓨팅(Quantum Computing)의 최신 발전**에 대한 종합적인 정리입니다. 각 항목에는 관련 출처를 명확히 인용했습니다.

---

##  1. 하드웨어 혁신 및 시스템 로드맵

- **QuantWare의 VIO‑40K 3D 배선 아키텍처**  
  네덜란드 QuantWare가 발표한 VIO‑40K 아키텍처는 수직 고밀도 배선과 칩렛 기반 설계로 최대 **10,000큐빗**까지 확장 가능한 양자 프로세서를 구현합니다. 이는 구글·IBM의 기존 시스템 대비 100배 높은 큐빗 수이며, 2028년 출하 예정이고, 2026년 네덜란드 델프트에 대형 양자 팹을 오픈할 계획입니다 ([livescience.com](https://www.livescience.com/technology/computing/breakthrough-3d-wiring-architecture-enables-10-000-qubit-quantum...
--------------------------------------------------

2️⃣ 에이전트: Research Assistant
질문: 전 세계 재생 에너지 도입 현황은 어떤가요?
--------------------------------------------------
응답: 아래는 2025년 12월 현재 기준으로 집계된, 전 세계 재생 에너지 도입 현황에 관한 최신 분석입니다.

---

##  1. 글로벌 재생에너지 용량 현황 (2024년 기준)

- **총 재생에너지 설비 용량**  
  2024년 말 기준 전 세계 재생에너지 발전 설비 용량은 약 4,448 GW(기가와트)

### 단일 검색 테스트

In [14]:
async def single_search_test(query: str, agent_type: str = "research"):
    """
    단일 검색 쿼리를 테스트한다.
    
    Args:
        query: 검색 쿼리
        agent_type: 'research' 또는 'news'
    """
    agent = research_agent if agent_type == "research" else news_agent
    
    print(f"🔍 단일 검색 테스트")
    print(f"에이전트: {agent.name}")
    print(f"쿼리: {query}")
    print("=" * 50)
    
    try:
        result = await Runner.run(agent, query)
        print(f"\n응답:")
        print(result.final_output)
    except Exception as e:
        print(f"에러: {str(e)}")

# 테스트 실행 (원하는 쿼리로 변경 가능)
await single_search_test("한국의 AI 스타트업 동향은 어떤가요?", "research")

🔍 단일 검색 테스트
에이전트: Research Assistant
쿼리: 한국의 AI 스타트업 동향은 어떤가요?

응답:
다음은 2025년 12월 12일(금) 기준으로 조사된, 최신 한국 AI 스타트업 생태계 및 관련 정책·산업 동향입니다. 여러 출처를 참고해 구조화하였습니다.

---

##  정부의 AI 스타트업 육성 및 제도적 지원

• 2025년 2월, 중소벤처기업부는 ‘AI 핵심·전환(AX) 스타트업 육성 프로그램’을 출범하였습니다. 이 프로그램은 슈퍼갭 스타트업 1000+ 프로젝트의 일환으로, 소형 언어모델(sLLM), 제조 AI, 바이오 AI, 콘텐츠 AI 분야의 고성장 AI 스타트업 5곳에 최대 1억5천만원, 그리고 전환 스타트업 15곳에 8천만원 규모의 협업 자금을 지원합니다.([koreatechdesk.com](https://www.koreatechdesk.com/south-korea-launches-ai-core-development-program-to-foster-high-growth-startups?utm_source=openai))  
• 이어 AI 슈퍼갭 챌린지(2025년 2월 말), AI 지역 핵심 산업 협력 프로젝트(2025년 4월) 등을 통해 글로벌 기업과 국내 스타트업 간 상용화·공동 연구가 추진되고 있습니다.([koreatechdesk.com](https://www.koreatechdesk.com/south-korea-launches-ai-core-development-program-to-foster-high-growth-startups?utm_source=openai))

---

##  투자·벤처 생태계의 성장세

• 서울의 스타트업 생태계는 글로벌 하락세에도 불구하고 강한 성장세를 보이고 있습니다. 2022~2024년간 생태계 가치 약 1,440억 달러, 2025년 3분기 AI 스타트업 투자액은 전분기 대비 152% 증가한 약 2.43조 원(약 18억 달러)에 달했습니다.([koreaproductpost.com](htt

---

## 도구 사용 모범 사례


1. 함수 도구 (@function_tool)
   - 명확한 독스트링 작성 - 에이전트가 도구 목적 이해
   - 타입 힌트 사용 - 매개변수와 반환 타입 명시
   - Pydantic 모델로 구조화된 반환값
   - 예외 처리로 안정성 확보

2. 호스팅 도구 (WebSearchTool 등)
   - 적절한 컨텍스트 크기 선택
   - 위치 설정으로 관련성 높은 결과
   - 속도 vs 상세함 트레이드오프 고려

3. 에이전트 지시문
   - 사용 가능한 도구 명시적으로 언급
   - 각 도구를 언제 사용해야 하는지 안내
   - 도구 결과 활용 방법 설명

4. 도구 조합
   - 여러 도구를 함께 사용하여 기능 확장
   - 함수 도구 + 호스팅 도구 조합 가능
   - 각 도구의 역할 명확히 구분

---

## 마무리

이 튜토리얼에서는 OpenAI Agents SDK의 도구 기초를 다루었다. 다음과 같은 내용을 학습하였다:

1. **기본 함수 도구**
   - @function_tool 데코레이터
   - Pydantic 모델을 활용한 구조화된 반환값
   - 독스트링을 통한 자동 문서화
   - 여러 도구를 가진 에이전트

2. **호스팅 도구**
   - WebSearchTool 구성 및 사용
   - user_location과 search_context_size 매개변수
   - 실시간 정보 검색
   - 다양한 검색 구성 비교

이러한 도구를 활용하면 에이전트의 기능을 크게 확장하여 외부 데이터 접근, 실시간 정보 검색, 복잡한 작업 수행이 가능한 시스템을 구축할 수 있다.